In [ ]:
# Imports
import os
# import json
import re
from unidecode import unidecode
from bs4 import BeautifulSoup as bs
from elasticsearch import Elasticsearch, helpers

In [ ]:
# Elastic host
es = Elasticsearch('https://localhost:9200')

es = Elasticsearch(
    hosts=[
            "https://localhost:9200"
    ],
    http_auth=("elastic", "NES9DZ-QwhanXAQf9caV"),
#     use_ssl=True,
    verify_certs=False#,
#     ca_certs="./ca.crt",
)

# Folder path
path = r"C:\Users\Asher\Documents\School\_Scriptie\Data\Data_Extract"

# Set directory
os.chdir(path)

# Print folder contents (should contain all .TEI folders)
os.listdir()

In [ ]:
# --------------------------------------------------------------------------------------
# Vereist list met folder van landen bv: ['ParlaMint-NL.TEI', "ParlaMint-BE.TEI"]
#
# ---Return list met dict [{Land1}, {Land 1}]---
#
# Voorbeeld voor land MET jaarfolders:
# item 1 --> 'land folder': [land xml] bv: 'ParlaMint-NL.TEI': ['ParlaMint-NL.xml']
# item rest --> 'ParlaMint-NL.TEI\\2014': ['ParlaMint-NL_2014-04-16-tweedekamer-2.xml', '...']
#
# Voorbeeld met land ZONDER jaarfolder:
# items (alle) --> {'ParlaMint-BG.TEI': ['ParlaMint-BG.xml', 'ParlaMint-BG_2014-10-27.xml', '...'}
#
# Ofwel len(dict) > 1 bij landen met jaarfolders, anders zit speciale file inbegrepen als eerste in dict
#
#  Path vinden met os.path.join(key, value[x])
# --------------------------------------------------------------------------------------

def get_xml_files(country_selection):
    country_return_list = []

    # loop door alle folders die hierboven zijn geprint
    for country in os.listdir():

        # filter op specifiek land (IN BOX 2)
        if country in country_selection:
            paths_dict = {}

            # ga door alle inhoud van de landfolder heen
            for root, dirs, files in os.walk(country):
                file_data = []
                
                # loop door files van een folder
                for file in files:

                    #filter alleen de xml files
                    if ".xml" in file and not "~" in file:
                        file_data.append(file)

                # filter onzin uit de dict voor makkelijkere processing later
                if not "Schema" in root:
                    paths_dict[root] = file_data

            country_return_list.append(paths_dict)
        
    return country_return_list

In [ ]:
# functie testruimte
selected_countries = ["ParlaMint-NL.TEI"]

answer = get_xml_files(selected_countries)

print(answer)

In [ ]:
# {
#     “#personname”: “Person Name”,
#     “MORE INFO”: “LATER” //TODO
# }

# haalt nodige informatie uit het overkoepelende bestand dat per land bestaat
# NOTE: Deze functie gaat later overige nodige info over personen en partijen ook meegeven
# TODO: https://stackoverflow.com/questions/33215515/use-character-like-%C3%AB-and-%C3%A7-in-python-file
# goede characteromvorming schrijven zodat puntjes op speciale letters niet vervormen
def extract_info_xml(root, file):
    path = os.path.join(root, file)
    person_dict = {}
    
     # lees het bestand
    with open(path, "r", encoding="utf-8") as file:
        # read each line in the file, readlines() returns a list of lines
        content = file.readlines()
        # combine the lines in the list into a string
        content = "".join(content)
        
        bs_content = bs(content, "lxml")
    
    # ga door alle user info heen
    persons = bs_content.find_all("person")
    for person in persons:
        person_dict[person.get("xml:id")] = (person.find('forename').text + " " + person.find('surname').text)
        
    return person_dict

In [ ]:
# functie testruimte
root, file = "ParlaMint-NL.TEI", "ParlaMint-NL.xml"

test_person_dict = extract_info_xml(root, file)

print(test_person_dict)

In [ ]:
# [{
#     "_index": "search",
#     "_id": int,
#     "_source": {
#         “prs”: “Geert Wilders”,
#         “seg”: “ParlaMint-NL_2017-01-17-eerstekamer-5.seg27”
#         “cnt”: “Nee, oneens”
#     }
# }, ...]

# haal de nodige data uit de xml en zet het in een list dict
def extract_debate_xml(root, file, persondict, cnt=0, index="search"):
    path = os.path.join(root, file)
    content_dict_list = []
    
    # lees het bestand
    with open(path, "r", encoding="utf-8") as file:
        # read each line in the file, readlines() returns a list of lines
        content = file.readlines()
        # combine the lines in the list into a string
        content = "".join(content)
        bs_content = bs(content, "lxml")
    
    # ga door alle segmenten info heen
    userlines = bs_content.find_all("u")
    for line in userlines:
        # vage oplossing tegen NONE lines, weet niet zeker of dit het echt oplost
        if line.get("who") == None:
            break
        
        content_dict, es_content_dict = {}, {}
        
        prs = persondict[line.get("who").replace('#', '')]
        
        # person, person_simplified, segment, content
        content_dict["prs"] = prs
        content_dict["prs_s"] = unidecode(prs)
        content_dict["seg"] = line.get("xml:id")
        content_dict["cnt"] = str(line.text).strip().replace('\n', ' ') # misschien \n voor SPACE filteren???
    
        # voeg nodige info toe aan elke xml_content_dict line voor bulk uploads
        cnt += 1
        
        es_content_dict["_index"] = index
        es_content_dict["_id"] = cnt
        es_content_dict["_source"] = content_dict
    
        # add the line to the list of lines
        content_dict_list.append(es_content_dict)
    
    return content_dict_list, cnt

In [ ]:
# functie testruimte
root, file = "ParlaMint-NL.TEI\\2017", "ParlaMint-NL_2017-01-17-eerstekamer-5.xml"
persondict = extract_info_xml("ParlaMint-NL.TEI", "ParlaMint-NL.xml")

test_json = extract_debate_xml(root, file, persondict)

print(test_json[0])

In [ ]:
# upload xml files (progressie)

# --1.0 upload xml in stukken
# id_count = 0

# for line in test_json:
#     id_count += 1
#     resp = es.index(index="search", id=id_count, document=json.dumps(line))

# --1.1 upload xml in BULK
# helpers.bulk(es, test_json)

# --1.2
# selected_countries = ["ParlaMint-BG.TEI"]

# countries = get_xml_files(selected_countries)
# for country in countries:
#     rootfolder = (k := next(iter(country)), country.pop(k))
    
#     # id counter voor elastic
#     cnt = 0
    
#     # verzamel person info
#     country_info = extract_info_xml(rootfolder[0], rootfolder[1][0])
#     print(rootfolder[1][0])
#     # jaarfolders
#     if len(rootfolder[1]) == 1:
        
#         for year_folder in country:
            

#             for file in country[year_folder]:
#                 processed_xml, cnt = extract_debate_xml(year_folder, file, country_info, cnt)
                
#                 helpers.bulk(es, processed_xml)

#     # geen jaarfolders (NOG NIET BEVESTIGD OF IE WERKT)
#     else:
        
#         rootfolder[1].pop(0)
        
#         for file in rootfolder[1]:
#             processed_xml, cnt = extract_debate_xml(rootfolder[0], file, country_info, cnt)
            
#             helpers.bulk(es, processed_xml)

In [ ]:
import time

In [ ]:
# zoek testruimte
start_time = time.time()

search_in = "cnt"
search = "stikstof"

resp = es.search(index="search", body={"query": {"bool" : {"must": {"match_phrase": {search_in: search,}},},},})
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print("%(prs)s %(seg)s:\n %(cnt)s \n" % hit["_source"])
    
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# verwijder inhoud "search" index
# es.options(ignore_status=[400,404]).indices.delete(index='search')